# Smart signatures – Transaction Fee Attack
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [3]:
from pyteal import *

In [4]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()["last-round"]

27685443

In [5]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU
CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
BY5K2AYO7R3G66ICY6SJ2JFVLRMIX677EAEEKDBTJZGP6Q4JVNZRDXDBKA


## Clearing out Modesty

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [6]:
max_amount = Int(int(1*1E6))                         # <---- 1e6 micro Algos = 1 Algo

modest_pyteal = And (
    Txn.receiver() == Addr(Bob["public"]),           # Receipient must be Bob
    Txn.amount() <= max_amount                       # Requested amount must be smaller than max_amount
)

# Security missing (!!!) ... do not copy-paste

##### Step 2: Compile PyTeal -> Teal

In [8]:
modest_teal = compileTeal(modest_pyteal, Mode.Signature, version=8)
print(modest_teal)

#pragma version 8
txn Receiver
addr CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
==
txn Amount
int 1000000
<=
&&
return


##### Step 3: Compile Teal -> Bytecode for AVM

In [9]:
Modest = algod_client.compile(modest_teal)
Modest

{'hash': '677HTEDHR5NL4YTX52ABGQLMHEDDCM5VR2OD5DH2GI2WSXW5RCTCBBOT3Q',
 'result': 'CDEHgCAT6T3nokbu5GeAjI1CwH+vMGwm3QlBll79w6vFJEpHmBIxCIHAhD0OEEM='}

##### Step 4: Alice funds and deploys the smart signature

In [10]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Modest['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27685460.
Waiting for round 27685460 to finish.
Waiting for round 27685461 to finish.
Transaction PIKC2QFDV6QR7YFS3IZNZYR7YJABSA5ZZA77PBQPAHTA4Q3JYCVA confirmed in round 27685462.


##### Step 5: Alice informs Bob

In [11]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Modest['result'])
print("Address of smart signature: ", Modest['hash'])

Alice communicates to Bob the following
Compiled smart signature: CDEHgCAT6T3nokbu5GeAjI1CwH+vMGwm3QlBll79w6vFJEpHmBIxCIHAhD0OEEM=
Address of smart signature:  677HTEDHR5NL4YTX52ABGQLMHEDDCM5VR2OD5DH2GI2WSXW5RCTCBBOT3Q


#### Step 6: Bob proposes a transaction with very high TX fee

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
sp.fee = int(2.38e6)                         # <---------- WOW! 2 ALGO transaction fee
sp.flat_fee = True

withdrawal_amt = int(0.00001*1e6)             # <---------- small
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Step 7: The Money is gone

In [13]:
# Check on Algoexplorer
print('https://testnet.algoexplorer.io/address/'+ Modest['hash'])

https://testnet.algoexplorer.io/address/677HTEDHR5NL4YTX52ABGQLMHEDDCM5VR2OD5DH2GI2WSXW5RCTCBBOT3Q
